In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pasc-data-quest-20-20/doctor_train.csv
/kaggle/input/pasc-data-quest-20-20/doctor_test.csv
/kaggle/input/pasc-data-quest-20-20/sample_submission.csv
/kaggle/input/derivedfff/Y_test.csv


In [51]:
X = pd.read_csv("/kaggle/input/pasc-data-quest-20-20/doctor_train.csv")
testF = pd.read_csv("/kaggle/input/pasc-data-quest-20-20/doctor_test.csv")
soln = pd.read_csv("/kaggle/input/derivedfff/Y_test.csv")

soln["Y"] = soln["Y"].replace(['yes'],1 )
soln["Y"] = soln["Y"].replace(['no'], 0 )
soln = soln["Y"].to_numpy()

testF['age'].fillna((testF['age'].mean()), inplace=True)
testF['Money'].fillna((testF['Money'].mean()), inplace=True)
testF["edu"].fillna("None", inplace = True) 
testF["prev_diagnosed"].fillna("None", inplace = True) 
testF["residence"].fillna("None", inplace = True) 
X['age'].fillna((X['age'].mean()), inplace=True)
X['Money'].fillna((X['Money'].mean()), inplace=True)
X["edu"].fillna("None", inplace = True) 
X["prev_diagnosed"].fillna("None", inplace = True) 
X["residence"].fillna("None", inplace = True) 
del testF["ID"]
del X["ID"]

X['age_Range'] = pd.cut(X['age'], bins=[0, 33, 40, 48, 100], labels=[1,2,3,4])

#X['Money_Range'] = pd.cut(X['Money'], bins=[-10000, 83, 487, 1400, 202127], labels=[1,2,3,4])
from sklearn import preprocessing
x = X[['Money']].values.astype(float)
minmaxScaler = preprocessing.MinMaxScaler()
X['MoneyTran'] = minmaxScaler.fit_transform(x)
x = testF[['Money']].values.astype(float)
testF['MoneyTran'] = minmaxScaler.fit_transform(x)
testF['MoneyTran'] = testF['MoneyTran'] + 0.00001
X['MoneyTran'] = X['MoneyTran'] + 0.00001

X['day_Range'] = pd.cut(X['day'], bins=[-1, 8, 16, 21, 35], labels=[1,2,3,4])
X['Time_Range'] = pd.cut(X['Time'], bins=[-1, 103, 179, 320, 40000], labels=[1,2,3,4])
X['DoctorVisits_Range'] = pd.cut(X['Doctor_visits'], bins=[0, 1, 2, 3, 58], labels=[1,2,3,4])
#del lastvisit, cured_in probably # check its valuse count below
X['curedIn_Range'] = pd.cut(X['cured_in'], bins=[-1, 0, 1, 3, 275], labels=[1,2,3,4])
X['lastVisit_Range'] = pd.cut(X['last_visit'], bins=[-20, 0, 150, 270, 1000], labels=[1,2,3,4])



testF['age_Range'] = pd.cut(testF['age'], bins=[0, 33, 40, 48, 100], labels=[1,2,3,4])
#testF['Money_Range'] = pd.cut(testF['Money'], bins=[-10000, 83, 487, 1400, 202127], labels=[1,2,3,4])
testF['day_Range'] = pd.cut(testF['day'], bins=[-1, 8, 16, 21, 35], labels=[1,2,3,4])
testF['Time_Range'] = pd.cut(testF['Time'], bins=[-1, 103, 179, 320, 40000], labels=[1,2,3,4])
testF['DoctorVisits_Range'] = pd.cut(testF['Doctor_visits'], bins=[0, 1, 2, 3, 58], labels=[1,2,3,4])
#del lastvisit, cured_in probably # check its valuse count below
testF['curedIn_Range'] = pd.cut(testF['cured_in'], bins=[-1, 0, 1, 3, 275], labels=[1,2,3,4])
testF['lastVisit_Range'] = pd.cut(testF['last_visit'], bins=[-20, 0, 150, 270, 1000], labels=[1,2,3,4])


#del the old columns
del X['age']
del X['Money']
del X['day']
del X['Time']
del X['Doctor_visits']
del X['cured_in']
del X['last_visit']

del testF['age']
del testF['Money']
del testF['day']
del testF['Time']
del testF['Doctor_visits']
del testF['cured_in']
del testF['last_visit']

X['age_Range'] = X['age_Range'].astype(int)
#X['Money_Range'] = X['Money_Range'].astype(int)
X['day_Range'] = X['day_Range'].astype(int)
X['Time_Range'] = X['Time_Range'].astype(int)
X['DoctorVisits_Range'] = X['DoctorVisits_Range'].astype(int)
X['curedIn_Range'] = X['curedIn_Range'].astype(int)
X['lastVisit_Range'] = X['lastVisit_Range'].astype(int)


testF['age_Range'] = testF['age_Range'].astype(int)
#testF['Money_Range'] = testF['Money_Range'].astype(int)
testF['day_Range'] = testF['day_Range'].astype(int)
testF['Time_Range'] = testF['Time_Range'].astype(int)
testF['DoctorVisits_Range'] = testF['DoctorVisits_Range'].astype(int)
testF['curedIn_Range'] = testF['curedIn_Range'].astype(int)
testF['lastVisit_Range'] = testF['lastVisit_Range'].astype(int)

X.Profession = pd.Categorical(X.Profession)
X['Profession'] = X.Profession.cat.codes
X.Month = pd.Categorical(X.Month)
X['Month'] = X.Month.cat.codes
X.Status = pd.Categorical(X.Status)
X['Status'] = X.Status.cat.codes
X.edu = pd.Categorical(X.edu)
X['edu'] = X.edu.cat.codes
X.Irregular = pd.Categorical(X.Irregular)
X['Irregular'] = X.Irregular.cat.codes
X.residence = pd.Categorical(X.residence)
X['residence'] = X.residence.cat.codes

#X.prev_diagnosed = pd.Categorical(X.prev_diagnosed)
#X['prev_diagnosed'] = X.prev_diagnosed.cat.codes
X['pd'] = 1
X.loc[X['prev_diagnosed'] == 'no', 'pd'] = 5
del X['prev_diagnosed']
testF['pd'] = 1
testF.loc[testF['prev_diagnosed'] == 'no', 'pd'] = 5
del testF['prev_diagnosed']

X.communication = pd.Categorical(X.communication)
X['communication'] = X.communication.cat.codes
X.side_effects = pd.Categorical(X.side_effects)
X['side_effects'] = X.side_effects.cat.codes


testF.Profession = pd.Categorical(testF.Profession)
testF['Profession'] = testF.Profession.cat.codes
testF.Month = pd.Categorical(testF.Month)
testF['Month'] = testF.Month.cat.codes
testF.Status = pd.Categorical(testF.Status)
testF['Status'] = testF.Status.cat.codes
testF.edu = pd.Categorical(testF.edu)
testF['edu'] = testF.edu.cat.codes
testF.Irregular = pd.Categorical(testF.Irregular)
testF['Irregular'] = testF.Irregular.cat.codes
testF.residence = pd.Categorical(testF.residence)
testF['residence'] = testF.residence.cat.codes
#testF.prev_diagnosed = pd.Categorical(testF.prev_diagnosed)
#testF['prev_diagnosed'] = testF.prev_diagnosed.cat.codes
testF.communication = pd.Categorical(testF.communication)
testF['communication'] = testF.communication.cat.codes
testF.side_effects = pd.Categorical(testF.side_effects)
testF['side_effects'] = testF.side_effects.cat.codes

X['Y'] = X['Y'].replace(['yes'], 1)
X['Y'] = X['Y'].replace(['no'], 0)



AttributeError: 'DataFrame' object has no attribute 'prev_diagnosed'

In [50]:
X

,Profession,Status,edu,Irregular,residence,communication,Month,side_effects,Y,age_Range,MoneyTran,day_Range,Time_Range,DoctorVisits_Range,curedIn_Range,lastVisit_Range,loanTran
0,4,2,3,0,1,0,1,3,0,1,0.086123,2,2,4,1,1,5
1,8,2,2,0,1,2,11,3,0,2,0.075337,3,1,1,1,1,5
2,4,1,3,0,2,2,8,3,0,3,0.127831,2,1,2,1,1,5
3,1,1,1,0,1,0,5,3,0,1,0.078642,1,4,1,1,1,1
4,0,1,2,0,1,0,9,3,0,2,0.096400,3,2,1,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28191,1,2,3,0,2,0,5,3,0,2,0.075201,2,1,4,1,1,5
28192,9,1,2,0,2,0,8,1,0,4,0.072813,1,3,1,2,4,5
28193,1,2,2,1,2,2,6,3,0,1,0.067593,1,2,4,1,1,5
28194,0,1,2,0,2,0,8,3,0,2,0.098080,2,2,2,1,1,5


In [29]:
from sklearn.utils import resample
df_majority = X[X.Y==0]
df_minority = X[X.Y==1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=18000,    # to match majority class
                                 random_state=123)

df_upsampled = pd.concat([df_majority, df_minority_upsampled])
#df_upsampled.Y.value_counts()

df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=18000,     # to match minority class
                                 random_state=123)

df_downsampled = pd.concat([df_majority_downsampled, df_minority_upsampled])
df_downsampled.Y.value_counts()

1    18000
0    18000
Name: Y, dtype: int64

In [30]:
X_new = df_downsampled
Y_new = X_new["Y"]
del X_new["Y"]

X_resampled = X_new
y_resampled = Y_new

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=24)

X_train

,Profession,Status,edu,Irregular,residence,prev_diagnosed,communication,Month,side_effects,age_Range,Money_Range,day_Range,Time_Range,DoctorVisits_Range,curedIn_Range,lastVisit_Range
14491,7,1,3,0,1,1,0,1,3,4,4,1,3,2,1,1
9714,1,2,2,0,1,2,0,5,3,1,3,4,4,4,1,1
16630,9,1,1,0,1,2,0,5,3,2,1,4,3,1,1,1
26082,0,1,1,0,1,1,0,5,1,4,4,2,3,4,4,2
15330,1,1,2,0,2,1,2,8,3,3,4,2,4,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374,11,1,4,0,1,1,2,6,3,3,4,3,4,3,1,1
592,0,1,2,0,2,1,0,6,0,3,4,4,3,1,4,2
21373,5,0,2,0,1,1,0,8,3,4,4,2,4,2,1,1
21045,4,1,3,0,2,2,2,8,3,2,1,3,2,1,1,1


In [34]:
from xgboost import XGBClassifier
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from sklearn import neighbors
from sklearn.svm import SVR,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns


#model = SVR(kernel='rbf')

#model.fit(X_train, y_train)



#Create a svm Classifier
model = SVR(kernel='rbf', C=1.0) # Linear Kernel

#gamma : it decides linearity of classificaction/dividing curve , directyl proportional
#C : It controls the trade off between smooth decision boundary and classifying the training points correctly.
#degree : degree is a parameter used when kernel is set to ‘poly’.degree of the polynomial used to find the hyperplane to split the data

#Train the model using the training sets
model.fit(X_train, y_train)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [39]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, predictions))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, predictions))

Accuracy: 76.93%
Precision: 0.7550147098154587
Recall: 0.7911995515695067


In [41]:
#answer
y_pred = model.predict(testF)
predictions = [round(value) for value in y_pred]
#print(np.shape(predictions))

sample_submission = pd.read_csv("/kaggle/input/pasc-data-quest-20-20/sample_submission.csv")
sample_submission["Y"] = predictions

sample_submission["Y"] = sample_submission["Y"].replace([1], 'yes')
sample_submission["Y"] = sample_submission["Y"].replace([0], 'no')


sample_submission.to_csv('submissions.csv', header=True, index=False)
sample_submission.head()


accuracy = accuracy_score(soln, predictions)
print("FINAL Accuracy: %.2f%%" % (accuracy * 100.0))

print(sample_submission["Y"].value_counts())

FINAL Accuracy: 78.83%
yes    1550
no     1450
Name: Y, dtype: int64
